<a href="https://colab.research.google.com/github/tgalbaugh/pythonProject/blob/master/pythonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount, import etc

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
#drive.mount('/content/gdrive',force_remount=True)
#%cd '/content/gdrive/Shareddrives/LMS Admins/LMSData/lmsCSV/TimTest/'

#drive.flush_and_unmount()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()  #asks permissions every time?
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#from google.colab import auth
#auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from google.cloud import bigquery

import pandas as pd
import io
import numpy as np
import datetime as dt
import pytz as tz
from googleapiclient.http import MediaIoBaseDownload
import sys
from google.colab.data_table import DataTable


Mounted at /content/gdrive


Select CSV file

In [6]:
fileID = None #ensure fileID variable is clear
driveId = '1-CAE_U88NdSr17dN5L2JgHe_IRyg8OPy' #LMS Admins Shared Drive
def listCSVfunc(folderID,folderName):
    fileList =  drive.ListFile({'q':"'"+folderID+"' in parents and trashed=false and mimeType='text/CSV'"
                            ,'driveId': driveId
                            ,'corpora': 'drive'
                            ,'maxResults': 1
                            ,'orderBy': 'createdDate' #'createdDate desc' to make descending
                            ,'includeItemsFromAllDrives': True
                            ,'supportsAllDrives': True}).GetList()
    dfCSVlist = pd.DataFrame(fileList)#put file list into dataframe
    if dfCSVlist.empty:
        print (folderName,"is empty")
    else: 
        dfCSVlist['createdDate'] = dfCSVlist['createdDate'].astype('datetime64[ns]')#convert createDate to datetime
        fileID = dfCSVlist.at[dfCSVlist['createdDate'].idxmin(), 'id']#get fileID for the CSV with the oldest createdDate
        fileName = dfCSVlist.at[dfCSVlist['createdDate'].idxmin(), 'title']#get fileID for the CSV with the oldest createdDate
        fileCreatedDate = (dfCSVlist.at[dfCSVlist['createdDate'].idxmin(), 'createdDate'])#get createdDate for the CSV with the oldest createdDate
        print ("CSV found in",folderName,fileName,fileID,fileCreatedDate )
        return [fileID,fileName] 

print(fileID)


None


In [ ]:

#folderID = '1h0YHnhDFDDD-lm5Zy_6jskzk65W7Rd44' #TimTest
#folderID = '1NoxFC4nxw808p4-2QSVPJVr8atW54jWk' #uploadedCSV
#folderID = '1m4k7fk-1no3pbGX9REAcxiwKOlh8wzN6' #lmsCSV
#folderID = '1XUBZKl35zHWASQmNOfy5SkJHxw8TeCN3' #TwoDayPlus
#folderID = '19LI1uy0YuszOOJEaE2ANOgYYQkq7vd8U' #ProblemSolveHoldingPen
'''
if fileID == None:
  fileInfo = listCSVfunc('1m4k7fk-1no3pbGX9REAcxiwKOlh8wzN6',"lmsCSV")
  if fileInfo != None:
    fileID = fileInfo[0]
    fileName = fileInfo[1]
if fileID == None:
  fileInfo = listCSVfunc('1XUBZKl35zHWASQmNOfy5SkJHxw8TeCN3',"TwoDayPlus")
  if fileInfo != None:
    fileID = fileInfo[0]
    fileName = fileInfo[1]
'''

# For testing to ensure a file is found
if fileID == None:
  fileInfo = listCSVfunc('1h0YHnhDFDDD-lm5Zy_6jskzk65W7Rd44',"TimTest")
  if fileInfo != None:
    fileID = fileInfo[0]
    fileName = fileInfo[1]
'''
'''

'''
# Save for possible future use
print(fileID)
print(fileName)
fileURL = 'https://drive.google.com/file/d/'+fileID
print(fileURL)
fileSharingURL = fileURL+'/view?usp=sharing'
print(fileSharingURL)
filePathWindows = 'G:/Shared drives/LMS Admins/LMSData/uploadedCSV/'+fileName
print(filePathWindows)
filePathPython = '/content/drive/Shared drives/LMS Admins/LMSData/uploadedCSV/'+fileName
print(filePathPython)
'''
if fileID == None:
  print("     ")
  print("**********No CSV files found in any folder**********")
  print("*****************Ending Script**********************")
  print("     ")
  sys.exit()
 
if fileID == None:
  print("************************************************************************")
  print("If you see this it didn't stop")
  print("************************************************************************")